In [1]:
import findspark
findspark.init('/home/fede/spark-2.1.0-bin-hadoop2.7')

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [5]:
from pyspark.ml.regression import LinearRegression

In [7]:
file_path = '/home/fede/Ecommerce_Customers.csv'
data = spark.read.csv(file_path, inferSchema=True, header=True)

In [8]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [13]:
data.head(1)[0]

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [12]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [14]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [15]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [18]:
assembler =VectorAssembler(inputCols=['Avg Session Length','Time on App','Time on Website', 'Length of Membership']
                          ,outputCol='features')

In [19]:
output = assembler.transform(data)

In [20]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [21]:
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [22]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [23]:
final_data = output.select('features', 'Yearly Amount Spent')

In [24]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [25]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [28]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                368|
|   mean| 501.95434557441814|
| stddev|   80.3346277922041|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [29]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                132|
|   mean| 491.95318149931506|
| stddev|  76.21376879313212|
|    min| 302.18954780965197|
|    max|  712.3963268096637|
+-------+-------------------+



In [30]:
lr = LinearRegression(labelCol='Yearly Amount Spent',featuresCol='features')

In [31]:
lr_model = lr.fit(train_data)

In [32]:
test_results = lr_model.evaluate(test_data)

In [34]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -4.640525276937467|
|  6.360077796841779|
|  -22.1111202459889|
| 10.151288338883887|
|-0.7308788496878833|
| 3.7394097705795275|
|  3.877504330284296|
| -4.126655041123286|
| -8.418204977195217|
|  4.305649357258801|
| -18.15751503038905|
| 18.043178686478768|
|-26.523271242435726|
| -1.507114139452085|
| -17.91495950238624|
| 1.2260411810182177|
| -2.290651805177049|
|-10.351935046552285|
|-5.6252251373470585|
| -8.712070103611211|
+-------------------+
only showing top 20 rows



In [35]:
test_results.rootMeanSquaredError

9.939027869537938

In [36]:
test_results.r2

0.9828634427563709

In [37]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [38]:
unlabeled_data = test_data.select('features')

In [39]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.8364326747734...|
|[30.9716756438877...|
|[31.1239743499119...|
|[31.1695067987115...|
|[31.2606468698795...|
|[31.3584771924370...|
|[31.3662121671876...|
|[31.4252268808548...|
|[31.5261978982398...|
|[31.5316044825729...|
|[31.5702008293202...|
|[31.6005122003032...|
|[31.6739155032749...|
|[31.7216523605090...|
|[31.8164283341993...|
|[31.8293464559211...|
|[31.8530748017465...|
|[31.8648325480987...|
|[31.8745516945853...|
|[31.8854062999117...|
+--------------------+
only showing top 20 rows



In [40]:
predictions = lr_model.transform(unlabeled_data)

In [41]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.8364326747734...| 472.1424257039271|
|[30.9716756438877...|488.27853196005094|
|[31.1239743499119...| 509.0581740857547|
|[31.1695067987115...| 417.2052424634089|
|[31.2606468698795...|422.05751010663926|
|[31.3584771924370...| 491.4365406788959|
|[31.3662121671876...| 426.7113782262006|
|[31.4252268808548...| 534.8933736958852|
|[31.5261978982398...|  417.512731169533|
|[31.5316044825729...|432.20995637210376|
|[31.5702008293202...| 564.1030071717939|
|[31.6005122003032...|461.12967280461817|
|[31.6739155032749...|502.24833915231693|
|[31.7216523605090...| 349.2840407713247|
|[31.8164283341993...| 519.0374510060426|
|[31.8293464559211...|383.92629680695677|
|[31.8530748017465...|  461.575775267529|
|[31.8648325480987...|450.24321552336596|
|[31.8745516945853...|397.91046938361455|
|[31.8854062999117...| 398.8153430760867|
+--------------------+------------